In [1]:
import os
import pandas as pd

In [2]:
df = pd.read_csv("Secom.csv")

In [3]:
# 특징과 라벨 분리
X = df.drop('Y', axis = 1)
Y = df['Y']

In [4]:
# 학습 데이터와 평가 데이터 분할
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [5]:
# 특징이 매우 많음을 확인
Train_X.shape

(1175, 590)

In [6]:
# 클래스 불균형 확인 => 언더샘플링을 적용하기에는 부적절 
Train_Y.value_counts()

-1    1100
 1      75
Name: Y, dtype: int64

In [7]:
# 클래스 불균형 비율 계산
Train_Y.value_counts().iloc[0] / Train_Y.value_counts().iloc[-1]

14.666666666666666

In [8]:
# kNN을 사용한 클래스 불균형 테스트
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import *
kNN_model = KNN(n_neighbors = 11).fit(Train_X, Train_Y)
pred_Y = kNN_model.predict(Test_X)
print(recall_score(Test_Y, pred_Y))
print(accuracy_score(Test_Y, pred_Y))

# 재현율이 0%로 불균형이 심각한 수준이라 보임

0.0
0.9260204081632653


In [9]:
# imblearn 0.0 버전, imbalanced-learn 0.7.0 버전을 설치한다.
# 만약 fit_sample() 을 호출할 때 오류가 생겼다면 mbalanced-learn 이 0.8.0 버전으로 설치되어있을 수 있다
# 오버샘플링은 상대적으로 적은 클래스의 데이터를 뻥튀기시켜 늘리는 방식이다.
# 가장 가까운 k개의 샘플을 찾고 그 이운들간에 선을 그어 무작위 점을 생성한다.
# 완전히 동일하진 않지만 샘플들 사이의 특성들을 반영한 데이터가 생성된다.
# 오버피팅의 가능성이 높아지는 부작용도 있다 - 단점

from imblearn.over_sampling import SMOTE
# SMOTE 인스턴스 생성
oversampling_instance = SMOTE(k_neighbors = 3)

# 오버샘플링 적용
o_Train_X, o_Train_Y = oversampling_instance.fit_sample(Train_X, Train_Y)

# ndarray 형태가 되므로 다시 DataFrame과 Series로 변환 (남은 전처리가 없다면 하지 않아도 무방)
o_Train_X = pd.DataFrame(o_Train_X, columns = X.columns)
o_Train_Y = pd.Series(o_Train_Y)

In [10]:
# 비율이 1:1이 됨을 확인
o_Train_Y.value_counts()

-1    1100
 1    1100
Name: Y, dtype: int64

In [11]:
# 같은 모델로 다시 평가: 정확도는 감소했으나, 재현율이 크게 오름을 확인
kNN_model = KNN(n_neighbors = 11).fit(o_Train_X, o_Train_Y)
pred_Y = kNN_model.predict(Test_X)
print(recall_score(Test_Y, pred_Y))
print(accuracy_score(Test_Y, pred_Y))

0.4482758620689655
0.5306122448979592


In [12]:
# 정확도를 조금더 올리는 방식으로 over_sampling을 진행
from imblearn.over_sampling import SMOTE
# SMOTE 인스턴스 생성
# sampling_strategy 를 이용해 균형 비율을 조정
oversampling_instance = SMOTE(k_neighbors = 3, sampling_strategy = {1:int(Train_Y.value_counts().iloc[0] / 2),
                                                                    -1:Train_Y.value_counts().iloc[0]})

# 오버샘플링 적용
o_Train_X, o_Train_Y = oversampling_instance.fit_sample(Train_X, Train_Y)

# ndarray 형태가 되므로 다시 DataFrame과 Series로 변환 (남은 전처리가 없다면 하지 않아도 무방)
o_Train_X = pd.DataFrame(o_Train_X, columns = X.columns)
o_Train_Y = pd.Series(o_Train_Y)

In [13]:
kNN_model = KNN(n_neighbors = 11).fit(o_Train_X, o_Train_Y)
pred_Y = kNN_model.predict(Test_X)
print(recall_score(Test_Y, pred_Y))
print(accuracy_score(Test_Y, pred_Y))

0.3103448275862069
0.6479591836734694
